In [ ]:
import os
import myServices as ms
import randForReg as rfr
import numpy as np
import pandas as pd
import sklearn
import joblib

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [44]:
DS = pd.read_csv('basin1CleanToTrain.csv', index_col = None)

In [45]:
print(DS.keys())
DS.drop(['Unnamed: 0'], axis = 1, inplace = True)
DS.info()
print(DS.keys())
DS.to_csv('basin1CleanToTrain2.csv',index=False)

Index(['Unnamed: 0', 'percentage', 'disToRiv', 'TWI', 'TPI', 'FAcc', 'slope',
       'elevation'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479538 entries, 0 to 2479537
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   percentage  int64  
 1   disToRiv    float64
 2   TWI         float64
 3   TPI         float64
 4   FAcc        float64
 5   slope       float64
 6   elevation   float64
dtypes: float64(6), int64(1)
memory usage: 132.4 MB
Index(['percentage', 'disToRiv', 'TWI', 'TPI', 'FAcc', 'slope', 'elevation'], dtype='object')


In [46]:
DS = pd.read_csv('basin1CleanToTrain2.csv', index_col = None)
print(DS.keys())

Index(['percentage', 'disToRiv', 'TWI', 'TPI', 'FAcc', 'slope', 'elevation'], dtype='object')


In [51]:
a ={}
a['noe'] ='baaa'
print(a)

{'noe': 'baaa'}
